# 5

**문제**: 2번의 문제를 “신경망 알고리즘”을 이용하여 해결하는 코드를 작성하여 제출하시오.  
다음 URL에 있는 코드를 실행하여 제출하면 됩니다.(한글 설명 추가시 가산점 있습니다.)  
https://pythonhealthcare.org/2018/04/17/73-machine-learning-neural-networks/

### 5.1 라이브러리 불러오기

Neural Network 학습을 위해 필요한 라이브러리 및 데이터를 불러온다.
- datasets: `sklearn` 라이브러리에서 기본적으로 제공하는 데이터세트를 불러온다.
- train_test_split: 데이터에서 training set과 test set을 나누어주는 함수로, 입력된 데이터를 무작위로 `training set`과 `test set`으로 나누며, 이 비율은 조절할 수 있다. 기본값으로는 `training`:`test` = 3:1로 나눈다.
- StandardScaler: feature에서 평균을 빼고 standard deviation으로 나눈다. 즉 feature를 unit variance로 scaling한다.
- pandas: 데이터를 표 형식 (DataFrame 형식)으로 수정할 수 있게 해 주는 라이브러리.
- numpy: Python의 scientific computing 라이브러리.
- matplotlib: Python의 시각화 라이브러리

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 5.2 함수화

문제 2에서 사용했던 코드 및 향수 사용할 데이터 시각화 코드 등을 추후 재사용을 위하여 함수화한다.

In [2]:
def load_data ():
    """Breast Cancer Wisconsin (Diagnostic) 데이터를 불러오는 함수. 데이터의 상세사항은 다음과 같다:
        .data (feature data를 포함하는 attribute)
        .feature_names (feature들의 이름을 포함하는 attribute)
        .target_names (판별 대상의 이름. Malignant (악성)과 Benign(양성)이 들어있음.)
        .target (0,1로 구성된 판별 대상의 label)
        .DESCR (데이터 세트의 상세사항에 대한 특성)"""
    
    data_set = datasets.load_breast_cancer()
    return data_set

def normalise (X_train,X_test):
    """X 데이터를 normalize한다. 즉, training set의 mean은 0이 되고, 편차는 1이 된다."""
    
    sc=StandardScaler() 
    sc.fit(X_train)
    X_train_std=sc.transform(X_train)
    X_test_std=sc.transform(X_test)
    return X_train_std, X_test_std

def split_data (data_set, split=0.25):
    """X, y 데이터를 비율 split만큼 training set과 test set으로 나눈다. 기본 split 비율은 0.25"""
    
    X=data_set.data
    y=data_set.target
    X_train,X_test,y_train,y_test=train_test_split(
        X,y,test_size=split)
    return X_train,X_test,y_train,y_test

def test_model(model, X, y):
    """입력값 X에 대해 y를 return함"""
    
    y_pred = model.predict(X)
    test_results = np.vstack((y, y_pred)).T
    return test_results

def train_model (X, y):
    """ Neural Network 모델을 학습시킴. """
    from sklearn.neural_network import MLPClassifier
    model = MLPClassifier(solver='lbfgs', alpha=1e-8, hidden_layer_sizes=(50, 5),
                        max_iter=100000, shuffle=True, learning_rate_init=0.001,
                        activation='relu', learning_rate='constant', tol=1e-7,
                        random_state=0)
    model.fit(X_train_std, y_train)   
    return model

def calculate_diagnostic_performance (actual_predicted):
    """ 모델의 성적을 산출한다.
    
    2개 column, 즉 실제값과 예측값으로 이루어진 Numpy array를 입력값으로 받는다.
    
    결과값을 dictionary 형태로 반환한다.:
        
    1) accuracy: proportion of test results that are correct    
    2) sensitivity: proportion of true +ve identified
    3) specificity: proportion of true -ve identified
    4) positive likelihood: increased probability of true +ve if test +ve
    5) negative likelihood: reduced probability of true +ve if test -ve
    6) false positive rate: proportion of false +ves in true -ve patients
    7) false negative rate:  proportion of false -ves in true +ve patients
    8) positive predictive value: chance of true +ve if test +ve
    9) negative predictive value: chance of true -ve if test -ve
    10) precision = positive predictive value 
    11) recall = sensitivity
    12) f1 = (2 * precision * recall) / (precision + recall)
    13) positive rate = rate of true +ve (not strictly a performance measure)
    """
    # Calculate results
    actual_positives = actual_predicted[:, 0] == 1
    actual_negatives = actual_predicted[:, 0] == 0
    test_positives = actual_predicted[:, 1] == 1
    test_negatives = actual_predicted[:, 1] == 0
    test_correct = actual_predicted[:, 0] == actual_predicted[:, 1]
    accuracy = np.average(test_correct)
    true_positives = actual_positives & test_positives
    true_negatives = actual_negatives & test_negatives
    sensitivity = np.sum(true_positives) / np.sum(actual_positives)
    specificity = np.sum(true_negatives) / np.sum(actual_negatives)
    positive_likelihood = sensitivity / (1 - specificity)
    negative_likelihood = (1 - sensitivity) / specificity
    false_positive_rate = 1 - specificity
    false_negative_rate = 1 - sensitivity
    positive_predictive_value = np.sum(true_positives) / np.sum(test_positives)
    negative_predictive_value = np.sum(true_negatives) / np.sum(test_negatives)
    precision = positive_predictive_value
    recall = sensitivity
    f1 = (2 * precision * recall) / (precision + recall)
    positive_rate = np.mean(actual_predicted[:,1])
    
    # Add results to dictionary
    performance = {}
    performance['accuracy'] = accuracy
    performance['sensitivity'] = sensitivity
    performance['specificity'] = specificity
    performance['positive_likelihood'] = positive_likelihood
    performance['negative_likelihood'] = negative_likelihood
    performance['false_positive_rate'] = false_positive_rate
    performance['false_negative_rate'] = false_negative_rate
    performance['positive_predictive_value'] = positive_predictive_value
    performance['negative_predictive_value'] = negative_predictive_value
    performance['precision'] = precision
    performance['recall'] = recall
    performance['f1'] = f1
    performance['positive_rate'] = positive_rate

    return performance

def print_feaure_importances (model, features):
    print ()
    print ('Feature importances:')
    print ('--------------------')
    df = pd.DataFrame()
    df['feature'] = features
    df['importance'] = model.feature_importances_
    df = df.sort_values('importance', ascending = False)
    print (df)
    return

def print_diagnostic_results (performance):
    """Iterate through, and print, the performance metrics dictionary"""
    
    print('\nMachine learning diagnostic performance measures:')
    print('-------------------------------------------------')
    for key, value in performance.items():
        print (key,'= %0.3f' %value) # print 3 decimal places
    return

### 5.3 데이터세트 준비하기
**Breast Cancer Wisconsin (Diagnostic)** data set을 함수를 이용해 불러오고 구조를 확인한다.  

In [3]:
data_set = load_data()
print(data_set)

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]]), 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
 

In [4]:
print('Data fields of the dataset: ')
print(data_set.feature_names, len(data_set.feature_names))
print('\nClassification outcome: ')
print(data_set.target_names)

Data fields of the dataset: 
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension'] 30

Classification outcome: 
['malignant' 'benign']


데이터세트의 'data'에 feature가 있고, target은 0, 1로 구분된 모습을 확인할 수 있다.  
총 30개의 features가 있으며, 이를 바탕으로 악성 (malignant, 0), 양성 (benign, 1) 종양을 구별하는 것이 본 코드의 목적이다.  

### 5.4 Neural Network 학습
데이터를 test set과 training set으로 train:test = 3:1의 비율로 나눈다.  
Neural Network의 경우 데이터의 정규화가 필요하다.  
이후 Random Forest Classifier 모델을 training set을 이용해 학습시키고 test set을 predict한 결과값을 `test_result`에 반환받는다.

In [6]:
# 데이터를 trainig set과 test set으로 나눈다.
X_train,X_test,y_train,y_test = split_data(data_set, 0.25)

# 데이터를 정규화시킨다.
X_train_std, X_test_std = normalise(X_train,X_test)

# 정규화된 데이터를 바탕으로 모델을 학습시킨다.
model = train_model(X_train_std,y_train)

# test set을 predict한 결과를 반환한다.
test_results = test_model(model, X_test_std, y_test)

### 5.5 학습 결과:
Neural Network 모델의 학습 결과를 바탕으로 모델의 성능을 출력한다.

In [8]:
# 결과를 바탕으로 모델의 성능을 계산한다
performance = calculate_diagnostic_performance(test_results)

# 모델의 성능을 출력한다.
print_diagnostic_results(performance)


Machine learning diagnostic performance measures:
-------------------------------------------------
accuracy = 0.979
sensitivity = 1.000
specificity = 0.953
positive_likelihood = 21.333
negative_likelihood = 0.000
false_positive_rate = 0.047
false_negative_rate = 0.000
positive_predictive_value = 0.963
negative_predictive_value = 1.000
precision = 0.963
recall = 1.000
f1 = 0.981
positive_rate = 0.573
